In [1]:
import pandas as pd
import numpy as np
import os
import datetime
from sklearn.linear_model import LinearRegression

In [10]:
pf = pd.read_csv(r'\\DEN-FS01\Home\kzimmerman\Desktop\df.csv')
pf.head()

,agcy,cpn,bb,px,sd
0,FGCI,2.0,G0,94.140625,12/18/2018
1,FGCI,2.5,G0,96.781250,12/18/2018
2,FGCI,3.0,G0,98.843750,12/18/2018
3,FGCI,3.5,G0,100.546875,12/18/2018
4,FGCI,4.0,G0,101.953125,12/18/2018


## F30

In [11]:
def model(df, delta):
    df.sd = pd.to_datetime(df.sd)
    today = pd.to_datetime(datetime.date.today())
    day = np.timedelta64(1,'D')
    df['date_delta'] = df.sd.sub(today).div(day)
    
    y = df[['px']].values
    X = df[['date_delta']].values
    return np.squeeze(LinearRegression().fit(X, y).predict(delta))

def group_predictions(df, dd):
    #sd = pd.to_datetime(sd)
    df.sd = pd.to_datetime(df.sd)
    day = np.timedelta64(1, 'D') # timedelta
    today = pd.to_datetime(datetime.date.today())
    #mn = df.sd.min()             # timestamp
    df['date_delta'] = df.sd.sub(today).div(day) # float
    
    dd = float(dd) # float
        
    return df.groupby(['agcy','cpn']).apply(model, delta=dd)

In [19]:
fwd = group_predictions(pf,30)
fwd = pd.DataFrame(fwd,index=fwd.index)
fwd = (fwd.assign(bb=lambda x: 'F30')
      .assign(sd=lambda x: datetime.date.today() +pd.Timedelta(30,unit='d'))
      .rename(columns={fwd.columns[0]:'px'}))
fwd.reset_index(inplace=True)
#del fwd['data_delta']
fwd.head()

,agcy,cpn,px,bb,sd
0,FGCI,2.0,94.08854166666667,F30,2019-01-12
1,FGCI,2.5,96.76822916666667,F30,2019-01-12
2,FGCI,3.0,98.81770833333333,F30,2019-01-12
3,FGCI,3.5,100.48177083333333,F30,2019-01-12
4,FGCI,4.0,101.80989583333333,F30,2019-01-12


In [20]:
delay=18
fwd['px']= fwd['px']+(fwd['cpn']*delay/360)
fwd.head()

,agcy,cpn,px,bb,sd
0,FGCI,2.0,94.1885,F30,2019-01-12
1,FGCI,2.5,96.8932,F30,2019-01-12
2,FGCI,3.0,98.9677,F30,2019-01-12
3,FGCI,3.5,100.657,F30,2019-01-12
4,FGCI,4.0,102.01,F30,2019-01-12


In [27]:
#del pf['date_delta']
data = pd.concat([pf,fwd],sort=True)
data.head()

,agcy,bb,cpn,px,sd
0,FGCI,G0,2.0,94.1406,2018-12-18 00:00:00
1,FGCI,G0,2.5,96.7812,2018-12-18 00:00:00
2,FGCI,G0,3.0,98.8438,2018-12-18 00:00:00
3,FGCI,G0,3.5,100.547,2018-12-18 00:00:00
4,FGCI,G0,4.0,101.953,2018-12-18 00:00:00


In [34]:
df = data

## Curr cpn pred

In [167]:
def linterp(df,y,x,pred):
    y=df[[y]]
    x=df[[x]]
    pred = pd.to_numeric(pred)
    return np.squeeze(LinearRegression().fit(x, y).predict(pred))

In [177]:
df.groupby('agcy').apply(linterp,'cpn','px',100)

agcy
FGCI     3.3919199382741922
FGLMC     3.814854378975543
FNCI      3.376043184155236
FNCL     3.8636779691443763
G2SF     3.6776497725552524
dtype: object

In [148]:
cc = df.groupby('agcy').apply(linterp,'cpn','px',100)

In [178]:
cd = df.groupby('agcy').apply(linterp,'cpn','px',100)
cd

agcy
FGCI     3.3919199382741922
FGLMC     3.814854378975543
FNCI      3.376043184155236
FNCL     3.8636779691443763
G2SF     3.6776497725552524
dtype: object

## High/Low cpn

In [112]:
def highc(x, base=0.5):
    return round(base*math.ceil(x/base),2)

In [97]:
def lowc(x, base=0.5):
    return round(base*round(x//base),2)

In [114]:
lowc(3.4222),highc(3.42222)

(3.0, 3.5)

In [159]:
highc(cc.iloc[1])

4.0

In [109]:
math.ceil(3.1/.5)*.5

3.5

In [111]:
(3.1//.5)*.5

3.0

In [165]:
for c in cc:
    highc = [c]
    round(0.5*math.ceil(c/0.5),2)

In [152]:
cc.shape, type(cc), cc.dtypes

((5,), pandas.core.series.Series, dtype('O'))